Quantum Convolutional Neural Network

This is a circuit that requires a connectivity map where the distance between the qubits increases exponentially with depth

In [1]:
from qiskit import QuantumCircuit

import time
import numpy as np

Define QCNN circuit

In [2]:
num_qubits = 4
num_layers = int(np.ceil(np.log2(num_qubits)))

qc=QuantumCircuit(num_qubits)
i_conv=0
for i_layer in range(num_layers):
    for i_sub_layer in [0 , 2**i_layer]:            
        for i_q1 in range(i_sub_layer, num_qubits, 2**(i_layer+1)):
            i_q2=2**i_layer+i_q1
            if i_q2<num_qubits:
                qc.rxx(np.random.rand(), i_q1, i_q2)
                qc.ry(np.random.rand(), i_q1)
                qc.ry(np.random.rand(), i_q2)
                i_conv+=1

Optionally print out the circuit

In [3]:
qc.draw()

┌───────────────┐┌─────────────┐                                »
q_0: ┤0              ├┤ Ry(0.72996) ├────────────────────────────────»
     │  Rxx(0.67581) │├─────────────┤┌───────────────┐ ┌────────────┐»
q_1: ┤1              ├┤ Ry(0.68052) ├┤0              ├─┤ Ry(0.5245) ├»
     ├───────────────┤├─────────────┤│  Rxx(0.86189) │┌┴────────────┤»
q_2: ┤0              ├┤ Ry(0.60573) ├┤1              ├┤ Ry(0.98939) ├»
     │  Rxx(0.75475) │├─────────────┤└───────────────┘└─────────────┘»
q_3: ┤1              ├┤ Ry(0.22804) ├────────────────────────────────»
     └───────────────┘└─────────────┘                                »
«     ┌───────────────┐┌─────────────┐
«q_0: ┤0              ├┤ Ry(0.15795) ├
«     │               │└─────────────┘
«q_1: ┤  Rxx(0.11414) ├───────────────
«     │               │┌─────────────┐
«q_2: ┤1              ├┤ Ry(0.11914) ├
«     └───────────────┘└─────────────┘
«q_3: ────────────────────────────────
«

Define backend

In [4]:
from qiskit.providers.fake_provider import GenericBackendV2

backend = GenericBackendV2(num_qubits=num_qubits, basis_gates = ['rz', 'rx', 'ry', 'cx'])

Compile with default passes

In [5]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
default_pm = generate_preset_pass_manager(backend=backend, optimization_level=3)

t1 = time.time()
default_qc = default_pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0))

Time taken:  0.05444645881652832
OrderedDict({'rz': 18, 'rx': 10, 'cx': 8, 'ry': 2})
Number of 2-qubit gates:  8
Number of 1-qubit gates:  30


In [6]:
# print(default_qc)

         ┌─────────┐ ┌─────────┐ ┌─────────┐               ┌───┐┌─────────────┐»
q_1 -> 0 ┤ Rz(π/2) ├─┤ Rx(π/2) ├─┤ Rz(π/2) ├───────────────┤ X ├┤ Rz(0.67581) ├»
         ├─────────┴┐├─────────┴┐└─────────┘               └─┬─┘└─────────────┘»
q_2 -> 1 ┤ Rz(-π/2) ├┤ Rx(-π/2) ├─────■──────────────────────┼─────────■───────»
         ├──────────┤└┬────────┬┘   ┌─┴─┐   ┌─────────────┐  │       ┌─┴─┐     »
q_3 -> 2 ┤ Ry(-π/2) ├─┤ Rz(-π) ├────┤ X ├───┤ Rz(0.75475) ├──┼───────┤ X ├─────»
         ├──────────┤┌┴────────┴┐   └───┘   └─────────────┘  │       └───┘     »
q_0 -> 3 ┤ Rz(-π/2) ├┤ Rx(-π/2) ├────────────────────────────■─────────────────»
         └──────────┘└──────────┘                                              »
«                        ┌───┐  ┌─────────┐  ┌─────────────┐     »
«q_1 -> 0 ───────────────┤ X ├──┤ Rz(π/2) ├──┤ Rx(0.68052) ├──■──»
«         ┌─────────────┐└─┬─┘  ├─────────┴┐ └─────────────┘┌─┴─┐»
«q_2 -> 1 ┤ Rx(0.60573) ├──┼────┤ Rz(-π/2) ├────────────────┤ X ├»
«  

UCC

In [10]:
#Test with UCC transpiler
import sys
sys.path.append('../')

from ucc import compile

t1 = time.time()
ucc_qc, gate_counts = compile(qc, mode="ucc", draw=False, get_gate_counts = True)
t2 = time.time()
print("Time taken: ", t2-t1)

# # Get gate counts
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cx", 0))
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

     ┌──────────┐┌────────┐                         ┌──────────────┐          »
q_0: ┤ Ry(-π/2) ├┤ Rz(-π) ├──■───────────────────■──┤ Ry(-0.72996) ├──────────»
     ├──────────┤├────────┤┌─┴─┐┌─────────────┐┌─┴─┐├──────────────┤          »
q_1: ┤ Ry(-π/2) ├┤ Rz(-π) ├┤ X ├┤ Rz(0.67581) ├┤ X ├┤ Ry(-0.68052) ├────■─────»
     ├──────────┤├────────┤└───┘└─────────────┘└───┘├──────────────┤  ┌─┴─┐   »
q_2: ┤ Ry(-π/2) ├┤ Rz(-π) ├──■───────────────────■──┤ Ry(-0.60573) ├──┤ X ├───»
     ├──────────┤├────────┤┌─┴─┐┌─────────────┐┌─┴─┐├─────────────┬┘┌─┴───┴──┐»
q_3: ┤ Ry(-π/2) ├┤ Rz(-π) ├┤ X ├┤ Rz(0.75475) ├┤ X ├┤ Ry(-1.7988) ├─┤ Rz(-π) ├»
     └──────────┘└────────┘└───┘└─────────────┘└───┘└─────────────┘ └────────┘»
«                                                                            »
«q_0: ────────────────────────────────────────────────■───────────────────■──»
«                         ┌─────────────┐ ┌────────┐  │                   │  »
«q_1: ─────────────────■──┤ Ry(-2.0953) ├─┤

Create custom compiler

In [8]:
# from qiskit.transpiler import PassManager
# import qiskit.transpiler.passes as passes 

# custom_pm = PassManager()

# custom_pm.append(passes.Collect2qBlocks())
# custom_pm.append(passes.ConsolidateBlocks())

# # custom_pm.append(passes.Decompose())
# # custom_pm.append(passes.Optimize1qGatesDecomposition())

# custom_qc = custom_pm.run(qc)

# gate_counts = custom_qc.count_ops()
# print(gate_counts)

# custom_qc.draw()

In [9]:
# # Get gate counts
# gate_counts = custom_qc.count_ops()
# print(gate_counts)
# print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0))
# print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))
